In [6]:
import autograd                       #prérequis au niveau de l'environement
from autograd import numpy as np
import matplotlib.pyplot as plt
from math import *



L'objectif de ce projet était le tracé informatique de lignes de niveaux.
Plus précisement, l'utilisateur saisit  f une fonction de deux variables à valeurs réelles, continuement différenciable, et le programme doit pouvoir représenter sur un graphique l'ensemble {(x,y), f(x,y)=c} avec c un réel donné en argument.
 Nous chercherons par plusieurs approches à remplir cette mission, avec deux contraintes principales:
 - Le temps de calcul qui ne doit pas excéder une durée d'attente raisonnable, compte tenu cependant de la complexité de la fonction à tracer, nous y renviendrons
 - La précision de la courbe rendue, qui là encore est soumis à la contrainte ci-dessus, ainsi qu'à la fonction.
 

Deux approches ont été envisagées, l'une naïve et l'autre plus élaborées, mais toutes deux reposent sur une méthode commune: le pavage de l'espace: on subdivise le plan en carrés de tailles égales, dont le côté sera un argument donné par l'utilisateur stocké dans la variable taille_cell, valant par défaut 1, nous reviendrons sur la necessité d'un choix adapté pour cette variable. 


Une première approche naïve: une approche par dichotomie uniquement.
Le principe global de cette méthode est le suivant: nous subdivisons chaque carré en 10 "tranches" de largeur (abscisses) 0.1taille_cell et de hauteur taille_cell.
Puis, sur chacune des colonnes, nous appliquons un algorithme de dichotomie à la fonction g qui, pour une abscisse abs_x fixée, donne à y f(abs_x,y)-c. Nous trouvons alors un point de la ligne de niveau, si il existe, sur chacune des colonnes, puis on relie tous ces points, "dans bon ordre", à la fin.

Voici l'algorithme qui trouve un point de la ligne de niveau sur une colonne donnée, et qui renvoit None si il n'en trouve pas. On demandera au lecteur d'ignorer les deux fonctions qui suivent, qu'il garde en mémoire pour l'instant qu'elles généralisent la recherche de graine sur les quatre côtés du carré (gauche: 1 puis on tourne dans le sens trigonométrique).


In [41]:

def find_seed(f,c=0,abs_x=0,ord_y=0,taille_cell=(1,1),eps=2**(-26)):
    def g(x,y):
        return(f(x,y)-c)
    a=ord_y
    b=ord_y+taille_cell[1]
    i=0
    while g(abs_x,a)>eps or abs(a-b)>eps:    #on cherche un point pas loin du zéro et dont la valeur n'est pas trop lointaine de zéro
        i+=1
        m=(a+b)/2
        if g(abs_x,m)*g(abs_x,a)<0:
            b=m
        else:
            a=m
    return a
    if i==int(log2(1/eps))+2:
          #si on a toujours pas convergé vers un point après avoir fait des découpages durant la dichotomie qui nous aurait amené à moins de eps du zero potentiel, on considère qu'il n'y a pas de zéro. Cette hypothèse repose sur le fait que la fonction est assez régulière.
        return "None"




def find_seed_x(f,abs_x,ord_y,taille_cell=(1,1),c=0,eps=2**(-26)):
    def g(x,y):
        return(f(x,y)-c)
    a=abs_x
    b=abs_x+taille_cell[0]
    i=0
    while g(a,ord_y)>eps or abs(a-b)>eps:       #on cherche selon les x
        i+=1
        m=(a+b)/2
        if g(m,ord_y)*g(a,ord_y)<0:
            b=m
        else:
            a=m
    return a
    if i==int(log2(1/eps))+2:

        return "None"





def find_seed_side(f,abs_x,ord_y,side,taille_cell=(1,1),c=0,eps=2**(-26)):
    if side==1:
        return find_seed(f,c,abs_x,ord_y,taille_cell,eps)
    elif side==2:
        return find_seed_x(f,c,abs_x,ord_y,taille_cell,eps)
    elif side==3:
        return find_seed(f,c,abs_x+taille_cell[0],ord_y,taille_cell,eps)
    else:
        return find_seed_x(f,c,abs_x,ord_y+taille_cell[1],taille_cell,eps)


On notera que le succès de l'algorithme de dichotomie, qui nécessite la présence d'une fonction monotone et assez régulière sur la colonne, avec un zéro unique, repose sur un pavage assez fin du plan pour que localement les lignes de niveaux puissent être approximées par des segments. 

Une première approche naïve du problème consiste à découper chaque cellule en 10 rectangles de largeur (abscisses) 0.1taille_cell et de hauteur taille_cell. 
Sur chacune de ses colonnes d'abscisse abs_x, on fait une recherche par dichotomie d'un point de la courbe de niveau pour y trouver un point si il existe. Puis, on les relie entre eux pour avoir le segment de la ligne de niveau. On comprend bien que tout le succès de la méthode repose sur le fait que les cellules sont assez petites, on est en effet en grand danger si deux portions de la ligne de niveaux qui se se touchent pas passent par cette cellule (en plus du fait qu'il faut assurer le succès de la recherche dichotomique)

In [2]:

def simple_contour(f, c=0.5, delta=0.01): # dans sa version naïve, il suffit d'effectuer un find_seed en changeant l'abscice x
    X=[0]                                 # le déplaçant de delta à chaque itération
    Y=[find_seed(f, c=c)]
    if Y[0]==None:
        return [],[]
    
    n=int(1//delta)
    for k in range(1,n+2):            #nous effectuons n+1 itérations
        A=find_seed(f, c=c, x=X[-1])
        if not A==None:
            X.append(X[-1]+delta)
            Y.append(A)
            
    return np.array(X), np.array(Y)

X,Y=simple_contour(cercle)  #nous effetuons un test sur un morceau de la fonction restreint [0,1]**2

#plt.plot(X,Y)
#plt.show()


NameError: name 'cercle' is not defined

Puis, on généralise le tracé à tout le plan en faisant des tracés dans chaque cellule, en faisant "tourner" les cellules de façon à conserver une fonction contour simple dont la graine, si elle existe, est dans sur le côté gauche du carré.

In [4]:
# Rotators
# ------------------------------------------------------------------------------
LEFT, UP, RIGHT, DOWN = 0, 1, 2, 3  # clockwise


def rotate_direction(direction, n=1):
    return (direction + n) % 4


def rotate(x, y, n=1):
    if n == 0:
        return x, y
    elif n >= 1:
        return rotate(1 - y, x, n - 1)
    else:
        assert n < 0
        return rotate(x, y, n=-3 * n)


def rotate_function(f, n=1):
    def rotated_function(x, y):
        xr, yr = rotate(x, y, -n)
        return f(xr, yr)

    return rotated_function


# Complex Contouring
# ------------------------------------------------------------------------------

# Customize the simple_contour function used in contour :
# simple_contour = smart_simple_contour


def contour(f, c, xs=[0.0, 1.0], ys=[0.0, 1.0], delta=0.01):
    curves = []
    nx, ny = len(xs), len(ys)
    for i in range(nx - 1):
        for j in range(ny - 1):
            xmin, xmax = xs[i], xs[i + 1]
            ymin, ymax = ys[j], ys[j + 1]

            def f_cell(x, y):
                return f(xmin + (xmax - xmin) * x, ymin + (ymax - ymin) * y)

            done = set()
            for n in [0, 1, 2, 3]:
                if n not in done:
                    rotated_f_cell = rotate_function(f_cell, n)
                    x_curve_r, y_curve_r = simple_contour(rotated_f_cell, c, delta)
                    exit = None
                    if len(x_curve_r) >= 1:
                        xf, yf = x_curve_r[-1], y_curve_r[-1]
                        if xf == 0.0:
                            exit = LEFT
                        elif xf == 1.0:
                            exit = RIGHT
                        elif yf == 0.0:
                            exit = DOWN
                        elif yf == 1.0:
                            exit = UP
                    if exit is not None:  # a fully successful contour fragment
                        exit = rotate_direction(exit, n)
                        done.add(exit)

                    x_curve, y_curve = [], []
                    for x_r, y_r in zip(x_curve_r, y_curve_r):
                        x, y = rotate(x_r, y_r, n=-n)
                        x_curve.append(x)
                        y_curve.append(y)
                    x_curve = np.array(x_curve)
                    y_curve = np.array(y_curve)
                    curves.append(
                        (xmin + (xmax - xmin) * x_curve, ymin + (ymax - ymin) * y_curve)
                    )
    for x, y in level_curves:
        matplotlib.pyplot.plot(x, y)
        
def cercle(x,y):
    return x**2+y**2
contour(cercle,0.5,[-2.0,-1.0,0.,1.,2.],[-2.0,-1.0,0.,1.,2.])

TypeError: find_seed() missing 2 required positional arguments: 'abs_x' and 'ord_y'

On constate donc que cette méthode, bien que rudimentaire, propose des résultats satisfaisants, mais avec une quantité gigantesque de calculs, et une approche assez rudimentaire. Cherchons donc une solution plus avancée, aussi bien sur le plan mathématique et informatique

Méthode de Newton: 

Principe: on pave l'espace, et l'on va comme dans la première méthode trouver un point de la courbe de niveau sur le côté gauche du carré.
Puis, on cherche à tracer le segment de courbe de niveau, si il existe, contenu dans le carré, les points étant séparé d''environ delta, avec delta=01*taille_cell*
Le principe est le suivant: on définit la fonction F qui à (x,y) donne (f(x,y)-c, distance((x,y),(w,z))**2-delta**2) avec (w,z) un point de la ligne de niveau déjà trouvé, en argument de la fonction F.
Le but finalement est de trouver un zéro à cette fonction: un point de la ligne de niveau à une distance delta du précédent.

In [1]:
def distance_carre(x,y,w,z):     
    return (z-y)**2+(w-x)**2

def F(f,x,y,w,z,c=0):
    return np.arrray([f(x,y)-c,distance_carre(x,y,w,z)-delta**2])

Explication mathématique de la méthode: nous définissons Fx et Fy les composantes en x et y de F
 
Un developpement limité au premier ordre donne, avec (x0,y0) un point du plan: 

$ F_x(x,y) = F_x(x_0,y_0)+ \frac{\partial F_x}{\partial x}(x_0,y_0)(x-x_0) + \frac{\partial F_x}{\partial y}(x_0,y_0)(y-y_0) + o(x-x_0,y-y_0)$

$ F_y(x,y) = F_y(x_0,y_0)+ \frac{\partial F_y}{\partial x}(x_0,y_0)(x-x_0) + \frac{\partial F_y}{\partial y}(x_0,y_0)(y-y_0) + o(x-x_0,y-y_0)$

Nous conserverons cette approximation.

Si on veut converger vers un point (x,y) qui annule F, nous tirons des deux lignes précédentes une relation: 

$\begin{pmatrix}
- F_x(x_0,y_0) \\
- F_y(x_0,y_0)
\end{pmatrix}
= J(x_0,y_0)
\begin{pmatrix}
x_1 - x_0\\
y_1-y_0
\end{pmatrix}
$

Puis par récurrence, nous obtenons la formule de l'algorithme de Newton généralisée en dimension 2:

$\begin{pmatrix}
x_{n+1}\\
y_{n+1}
\end{pmatrix}
=
\begin{pmatrix}
x_{n}\\
y_{n}
\end{pmatrix}
- J^{-1}(x_n,y_n)
\begin{pmatrix}
F_x(x_n,y_n)\\
F_y(x_n,y_n)
\end{pmatrix}
$



In [35]:
def grad(f,x,y):
    g=autograd.grad
    return np.array([g(f,0)(x,y),g(f,1)(x,y)])


#On construit la jacobienne de F en un point (x,y), en gardant en mémoire que l'on part de (w,z) donnés en argument

    
def Jacobienne_F(f,x,y):
    j=autograd.jacobian
    return np.c_[j(f,0)(x,y),j(f,1)(x,y)]


def tang_norm(f,x,y,delta=0.01):
                                  #retourne un vecteur, de norme delta, orthogonal au gradient de f en (x,y), donc tangent à la ligne de niveau de f en (x,y)
    
    
    g=autograd.grad
    normed=delta/((g(f,0)(x,y)**2+g(f,1)(x,y)**2)**(0.5))
    return np.r_[g(f,1)(x,y)*normed,-g(f,0)(x,y)*normed]



    
#Trouve le point suivant  

def trouve_suivant(f,xi,yi,d,c,eps):

    def F(x,y):
        return np.array([f(x,y)-c,(x-xi)**2+(y-yi)**2-d**2])

    (x,y)=(xi,yi)+tang_norm(f,xi,yi,d)

    compteur=0 #compte le nombre d'itÃ©rations

    while F(x,y)[0]**2+F(x,y)[1]**2 > eps**2 and compteur<100000:
        J=Jacobienne_F(F,x,y)
        try:
            (x,y)=(x,y)-np.dot(np.linalg.inv(J),F(x,y))
        except np.linalg.LinAlgError :
            return (x,y)
        compteur+=1

    return (x,y)



Nous sommes maintenant près à mettre en place le programme qui tracera le segment de ligne dans le carré. Notons que ce programme s'arretera quand on sera proche des côtés haut, doite et bas de la cellule (on part de sur le côté gauche, et l'on suppose que le découpage du plan est suffisement fin pourque la ligne de niveau de f ne coupe qu'une seule fois ce côté).

Notons aussi qu'au départ du côté gauche du carré, il faut faire le saut vers l'intérieur de la cellule, d'où ces petits programmes spécifiques à la première recherche (encore une fois, ignorons pour l'instant le paramètre side qui vaut 1 par défaut dans simple_contour, on regarde donc le côté gauche).

In [34]:

def choose_direction_init_side(f,x,y,side):
    direction=normalise_dim2(np.array([-grad(f,x,y)[1],grad(f,x,y)[0]]))
    if side==1:
        if direction[0]>=0:
            return direction
        else:
            return -direction
    if side==3:
        if direction[0]<=0:
            return direction
        else:
            return -direction
    if side==2:
        if direction[1]>=0:
            return direction
        else:
            return -direction
    else:
        if direction[1]<=0:
            return direction
        else:
            return -direction
    
def trouve_suivant_init_side(f,xi,yi,d,c,side,eps):

    def F(x,y):
        return np.array([f(x,y)-c,(x-xi)**2+(y-yi)**2-d**2])

    (x,y)=(xi,yi)+choose_direction_init_side(f,xi,yi,side)*d

    compteur=0 #compte le nombre d'itÃ©rations

    while F(x,y)[0]**2+F(x,y)[1]**2 > eps**2 and compteur<1000:
        J=Jacobienne_F(F,x,y)
        try:
            (x,y)=(x,y)-np.dot(np.linalg.inv(J),F(x,y))
        except np.linalg.LinAlgError :
            return (x,y)
        compteur+=1

In [42]:
def simple_contour(f,abs_x,ord_y,c=0.0,side=1,taille_cell=(1,1),eps=2**(-26)):
    d=0.01*taille_cell[0]
    X,Y=np.array([]),np.array([])
    if side==1 or side==3:
        start_y=find_seed_side(f,abs_x,ord_y,side,taille_cell,c)
        start_x=abs_x
        print(start_x,start_y)
    else:
        start_x=find_seed_side(f,abs_x,ord_y,side,taille_cell,c)
        start_y=ord_y
    print(start_x,start_y)

    if isinstance(start_y,float) and isinstance(start_x,float):
        X=np.append(X,start_x)
        Y=np.append(Y,start_y)
        (xi,yi)=trouve_suivant_init_side(f,start_x,start_y,d,c,side,eps)
        X=np.append(X,xi)
        Y=np.append(Y,yi)
        print(X,Y)

        compteur=0
        while xi>eps+abs_x and (abs_x+taille_cell[0]-eps)>xi and yi>eps+ord_y and (ord_y+taille_cell[1]-eps)>yi and compteur <1000:
            pt_prec=np.array([X[-2],Y[-2]])
            (xi,yi)=trouve_suivant(f,xi,yi,pt_prec,d,c,eps)
            X=np.append(X,xi)
            Y=np.append(Y,yi)
            compteur+=1
        print(X,Y)
        plt.plot(X,Y)
        plt.show()
    else:
        return (X,X)


In [43]:
def cercle(x,y):
    return x**2+y**2

simple_contour(cercle,0.,0.,0.5)

0.0 0.7071067690849304
0.0 0.7071067690849304


TypeError: cannot unpack non-iterable NoneType object